In [1]:
try:
    import osmnx as ox
except :
    # !pip install --break-system-packages osmnx
    !pip install osmnx
    import osmnx as ox 

In [2]:
# 광진구의 골목길 포함 도로 네트워크 가져오기 (residential, service, tertiary 도로 포함)
place_name = "광진구, 서울, 대한민국"
graph = ox.graph_from_place(place_name, network_type='drive')
# graph = ox.graph_from_place(place_name, network_type='bike')


In [3]:
# 모든 도로의 시작점과 종료점 노드 정보
nodes, edges = ox.graph_to_gdfs(graph)


In [4]:
print(f'{len(nodes)}')
print(f'{len(edges)}')

2671
7216


In [5]:
# edges 데이터프레임의 구조와 첫 몇 개의 행을 확인
print(edges.head())
print(edges.info())

# nodes 데이터프레임의 구조와 첫 몇 개의 행을 확인
print(nodes.head())
print(nodes.info())


                                                       osmid      highway  \
u         v           key                                                   
368702043 4104262755  0    [175320385, 175320419, 175320429]     tertiary   
          8963165264  0                            516647748  residential   
          436866074   0                            516647752  residential   
414683232 11125576999 0                           1199529109    secondary   
          11125576997 0                           1193259435      primary   

                                     name  oneway reversed      length tunnel  \
u         v           key                                                       
368702043 4104262755  0    [광장로, 광장앞지하차도]   False    False  430.542185    yes   
          8963165264  0              워커힐로   False    False  117.361840    NaN   
          436866074   0              워커힐로   False     True  177.203342    NaN   
414683232 11125576999 0              아차산로    True    Fa

In [6]:
# 전역 배열 초기화
road_names = []
highway_types = []
start_lats = []
start_lons = []
end_lats = []
end_lons = []
tunnel_status = []  # 빈 리스트로 초기화
bridge_status = []  # 빈 리스트로 초기화
service_status = []  # 빈 리스트로 초기화
road_ids = []
max_speeds = []
lane_counts = []
one_way = []
junction_status = []

# 도로 종류 한글 매핑 딕셔너리
road_type_map = {
    'motorway': '고속도로',
    'trunk': '간선도로',
    'primary': '주도로',
    'secondary': '2차로',
    'tertiary': '3차로',
    'unclassified': '비분류 도로',
    'residential': '주거지 도로',
    'service': '서비스 도로',
    'living_street': '생활도로'
}

# 'edges' 데이터프레임에서 각 도로에 대해 정보 추출
for index, row in edges.iterrows():
    # MultiIndex에서 u와 v 값 추출
    start_node = index[0]  # 시작 노드 (u)
    end_node = index[1]    # 종료 노드 (v)

    # 노드 정보를 기반으로 위도와 경도 가져오기
    try:
        start_lat, start_lon = nodes.loc[start_node, ['y', 'x']]
        end_lat, end_lon = nodes.loc[end_node, ['y', 'x']]
    except KeyError:
        # 노드 정보가 없을 경우 건너뛰기
        continue

    # 도로명(속성 name)과 도로 특성 확인
    road_name = row.get('name', None)  # 도로명이 없는 경우 None 처리
    highway_type = row.get('highway', None)  
    # if highway_type not in ['unclassified', 'motorway', 'trunk', 'primary', 'secondary', 'residential', 'tertiary',
    #                        'living_street', 'primary_link', 'trunk_link', 'secondary_link', 'road'] :
    #     print(highway_type)                                
    tunnel_status_val = row.get('tunnel', None)   
    bridge_status_val = row.get('bridge', None)  # 교량 여부
    service_status_val = row.get('service', None)  # 서비스 도로 여부
    maxspeed = row.get('maxspeed', None)  # 속도 제한
    lanes = row.get('lanes', None)  # 차선 수
    oneway = row.get('oneway', None)  # 일방통행 여부
    junction = row.get('junction', None)  # 교차로 여부

    # highway_type이 리스트인 경우 첫 번째 값만 사용
    if isinstance(highway_type, list):
        if(highway_type[0] == 'unclassified') :
            highway_type = highway_type[1]   
        else :
            highway_type = highway_type[0]   
        
    if not highway_type:  # None, 빈 문자열, 빈 리스트 등을 모두 포함
        continue

    if highway_type == 'unclassified' :
        continue
        
    # 국도나 고속도로 등을 제외
    if highway_type in ['motorway', 'trunk', 'primary', 'secondary', 'trunk_link', 'primary_link']:   
        continue  # 해당 도로는 건너뛰기


        
    # 전역 배열에 데이터 추가
    road_names.append(road_name if road_name else None)  # 값이 없으면 None로 처리
    # 도로 종류를 한글로 매핑
    road_type_in_korean = road_type_map.get(highway_type, None)
    highway_types.append(road_type_in_korean)  # 값이 없으면 None로 처리
    start_lats.append(start_lat if start_lat else None)  # 값이 없으면 None로 처리
    start_lons.append(start_lon if start_lon else None)  # 값이 없으면 None로 처리
    end_lats.append(end_lat if end_lat else None)  # 값이 없으면 None로 처리
    end_lons.append(end_lon if end_lon else None)  # 값이 없으면 None로 처리
    tunnel_status.append(tunnel_status_val if tunnel_status_val else None)  # 값이 없으면 None로 처리
    bridge_status.append(bridge_status_val if bridge_status_val else None)  # 값이 없으면 None로 처리
    service_status.append(service_status_val if service_status_val else None)  # 값이 없으면 None로 처리
    road_ids.append(row['osmid'] if row['osmid'] else None)  # 값이 없으면 None로 처리
    max_speeds.append(maxspeed if maxspeed else None)  # 값이 없으면 None로 처리
    lane_counts.append(lanes if lanes else None)  # 값이 없으면 None로 처리
    one_way.append(oneway if oneway else None)  # 값이 없으면 None로 처리
    junction_status.append(junction if junction else None)  # 값이 없으면 None로 처리


In [7]:
# 각 배열의 길이 확인
print(len(road_names), len(highway_types))
print(len(start_lats), len(start_lons))
print(len(end_lats), len(end_lons))


6288 6288
6288 6288
6288 6288


- dd
  

In [8]:
average_lat = []
average_lon = []

for i in range(max(len(start_lats), len(end_lats))):
    start_lat = start_lats[i] if i < len(start_lats) else start_lats[-1]
    end_lat = end_lats[i] if i < len(end_lats) else end_lats[-1]
    average_lat.append((start_lat + end_lat) / 2)

for i in range(max(len(start_lons), len(end_lons))):
    start_lon = start_lons[i] if i < len(start_lons) else start_lons[-1]
    end_lon = end_lons[i] if i < len(end_lons) else end_lons[-1]
    average_lon.append((start_lon + end_lon) / 2)


In [9]:
# 데이터프레임 생성
data = {
    '도로명': road_names,
    '도로 종류': highway_types,
    '시작점_위도': start_lats,
    '시작점_경도': start_lons,
    '종료점_위도': end_lats,
    '종료점_경도': end_lons,
    '중앙점_위도' : average_lat,
    '중앙점_경도' : average_lon,
    '터널 여부': tunnel_status,
    '교량 여부': bridge_status,
    '서비스 도로 여부': service_status,
    '도로 ID': road_ids,
    '속도 제한': max_speeds,
    '차선 수': lane_counts,
    '일방통행 여부': one_way,
    '교차로 여부': junction_status
}


In [10]:
import pandas as pd

df = pd.DataFrame(data)


In [11]:
df.to_csv('./make_file/광진구_자동차도로_전체.csv', index=False)

print("광진구의 도로 정보가 CSV 파일로 저장되었습니다.")


광진구의 도로 정보가 CSV 파일로 저장되었습니다.
